In [1]:
import logging
import hydra
import pandas as pd

from collections import defaultdict
from tqdm import tqdm 

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colormaps
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.patches import Ellipse

from erc import drawing_ellipse, split_df_by_gender


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset._target_=erc.datasets.KEMDy19Dataset"})


# select 1-fold 
cfg.dataset.mode = "train"
train_dataset = hydra.utils.instantiate(cfg.dataset)

cfg.dataset.mode = "valid"
cfg.dataset.validation_fold = 0
valid_dataset = hydra.utils.instantiate(cfg.dataset)

INFO:erc.utils:Instantiate KEMDy19 Dataset
INFO:erc.utils:./data/kemdy19.csv does not exists. Process from raw data


Processing ECG / EDA / Label from data/KEMDy19:   0%|          | 0/20 [00:00<?, ?it/s]

INFO:erc.utils:New dataframe saved as data/kemdy19.csv
INFO:erc.utils:Instantiate KEMDy19 Dataset


In [2]:
# Pre-training Scheme ... 
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ForSequenceClassification

pretrained_model = Wav2Vec2ForSequenceClassification.from_pretrained(
    # "wav2vec2-xls-r-300m-korean",
    "w11wo/wav2vec2-xls-r-300m-korean",
    num_labels=7
    )


Some weights of the model checkpoint at w11wo/wav2vec2-xls-r-300m-korean were not used when initializing Wav2Vec2ForSequenceClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at w11wo/wav2vec2-xls-r-300m-korean and are newly initialized: ['classifier.weight', 'projector.bias', 'projector.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [3]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from transformers import AdamW
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
from jiwer import wer # wer metircs 
from transformers import Wav2Vec2Processor

model = pretrained_model.to("cuda")
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = AdamW(model.parameters(), lr = 1e-5,  eps = 1e-8)

/home/hoesungryu/.conda/envs/etri-erc/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [4]:
import torch 
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size= 16)

total_loss = 0
train_acc_sum = 0
train_loss = []

for batch in train_loader: 
    optimizer.zero_grad()
    print(batch.keys())
    input_values = batch['wav']
    attention_mask = batch['wav_mask']
    print(input_values)
    print(attention_mask)
    
    # inputs = processor(batch["wav"], sampling_rate=16000, return_tensors="pt")
    # labels = batch['emotion'].to("cuda")
    
    # input_values = inputs.input_values.type(torch.FloatTensor).squeeze().to("cuda")
    # print(input_values.shape)
    break
    



RuntimeError: stack expects each tensor to be equal size, but got [1, 330697] at entry 0 and [1, 264639] at entry 1

In [ ]:
# from transformers import Wav2Vec2Processor, Wav2Vec2Model
# extractor = Wav2Vec2Model.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

In [11]:
def pad_value(
        arr: torch.Tensor,
        max_length: int,
        pad_value: int | float = 0,
    ):
        """ Assumes single data """
        if not isinstance(arr, torch.Tensor):
            arr = torch.tensor(arr)
        
        mask = torch.ones(max_length).long()
        if len(arr) >= max_length:
            arr = arr[:max_length]
        else:
            null_size = max_length - len(arr)
            arr = torch.nn.functional.pad(arr, pad=(0, null_size), value=pad_value)
            mask[null_size:] = 0
        return arr, mask


In [12]:

    def get_wav(self, wav_path: Path | str) -> torch.Tensor | np.ndarray:
        """ Get output feature vector from pre-trained wav2vec model
        XXX: Embedding outside dataset, to fine-tune pre-trained model? See Issue
        """
        wav_path = check_exists(wav_path)
        # sampling_rate, data = wavfile.read(wav_path)
        data, sampling_rate = torchaudio.load(wav_path)
        data, mask = self.pad_value(data, max_length=self.max_length_wav)
        return sampling_rate, data, mask

NameError: name 'Path' is not defined

In [24]:
import torchaudio 
path = "./data/KEMDy19/wav/Session01/Sess01_impro01/Sess01_impro01_F001.wav"
data, sampling_rate = torchaudio.load(path)
print(data.squeeze().shape)


torch.Size([49962])


In [23]:
from scipy.io import wavfile
sampling_rate, data = wavfile.read(path)
data = torch.tensor(data)
print(data.shape)

torch.Size([49962])


In [20]:
data.shape

torch.Size([1, 449960])

In [17]:
max_length_wav = 200_000
data, mask = pad_value(data, max_length=max_length_wav)

In [19]:
data.shape

torch.Size([1, 449960])

In [15]:
data, sampling_rate

(tensor([[ 0.0012,  0.0010, -0.0007,  ...,  0.0000,  0.0000,  0.0000]]), 16000)

In [ ]:
# def wav2processor()

# train_dataset.map()

In [ ]:


class EmotionClassfierWithAudio(nn.Module):
    def __init__(self,pretrained_model, n_classes=7):
        super().__init__()
        self.pretrianed_model = pretrained_model
        # for p in self.pretrianed_model.parameters():
            # p.requires_grad = False

        self.fc = nn.Linear(1025,n_classes)

    def forward(self, x):
        x = F.adaptive_avg_pool3d(self.pretrianed_model(x).logits , axis=1).squeeze()# mean of 355 // logit output shape: B, Seq(355), 1025
        x = self.fc(x)
        return x

In [ ]:
# pretrained_model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
pretrained_model(input_values).logits.shape

In [ ]:
model(input_values)

In [ ]:
a = extractor(batch["wav"].type(torch.FloatTensor))

In [ ]:
a.extract_features.shape

In [ ]:
a.extract_features[:,-1].shape

In [ ]:
F.adaptive_avg_pool1d(a.extract_features, output_size = 7).shape

In [ ]:

a.extract_features.

In [ ]:
model(input_values)

In [ ]:
inputs.input_values.shape

In [ ]:

    outputs = model(input_values)

    loss = criterion(outputs, labels)
    total_loss += loss.item()
    train_loss.append(total_loss/(step+1))

    loss.backward()
    optimizer.step()

avg_train_loss = total_loss / len(train_loader)
print(f'  Average training loss: {avg_train_loss:.2f}')


In [ ]:
# !pip install jiwer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# import soundfile as sf
import torch
from jiwer import wer # wer metircs 
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

batch = train_dataset[2]

inputs = processor(batch["wav"], sampling_rate=16000, return_tensors="pt", padding="longest")
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

input_values = inputs.input_values
print(input_values.shape)
with torch.no_grad():
    logits = model(input_values.type(torch.FloatTensor).to("cuda")).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(batch['txt'])
print(transcription)
print("WER:", wer(batch['txt'], transcription))

In [ ]:
logits.shape

In [ ]:
logits[:,-1].shape

In [ ]:
logits.shape

In [ ]:
logits.mean(axis=1).shape

In [ ]:
logits[:,-1]

In [ ]:
logits

In [ ]:
logits

In [ ]:
model(input_values.type(torch.FloatTensor).to("cuda"))

In [ ]:
predicted_ids